# Azure MLパイプラインにおける糖尿病データセットのリッジ回帰モデルのパラメータの実験

このノートブックは、糖尿病データセット上でリッジ回帰モデルを訓練するために、さまざまなパラメータを使って実験するためのものです。

In [ ]:
# 実験ディレクトリからの変更
%cd ..

In [ ]:
import azureml.core
from azureml.core import Workspace

In [ ]:
# 保存された設定ファイルからワークスペースを読み込む
ws = Workspace.from_config()

In [ ]:
import os, shutil

# 実験ファイル用のフォルダを作成する
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# 実験フォルダにデータファイルをコピー
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

# 実験フォルダにtrain関数をコピー
shutil.copy('diabetes_regression/training/train.py', os.path.join(training_folder, "train.py"))

In [ ]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "alpha": 0.3
    },
    "evaluation":
    {

    },
    "scoring":
    {
        
    }
}


In [ ]:
%%writefile $training_folder/diabetes_training.py
# ライブラリのインポート
from azureml.core import Run
import pandas as pd
import shutil
import joblib

from train import split_data, train_model

# パラメータの取得
parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', default="diabetes_model", help='output folder')
args = parser.parse_args()
output_folder = args.output_folder

# 実験の実行コンテキストの取得
run = Run.get_context()

# 糖尿病データセットの読み込み
print("Loading Data...")
train_df = pd.read_csv('diabetes.csv')

data = split_data(train_df)

# テストするパラメータを指定する
with open("parameters.json") as f:
    pars = json.load(f)
    train_args = pars["training"]

# パラメータのログ
for k, v in train_args.items():
    run.log(k, v)

model, metrics = train_model(data, train_args)

# メトリクスのログ
for k, v in metrics.items():
    run.log(k, v)

# パラメータファイルを出力フォルダに保存する
os.makedirs(output_folder, exist_ok=True)
shutil.copy('parameters.json', os.path.join(output_folder, 'parameters.json'))
joblib.dump(value=model, filename= output_folder + "/model.pkl")
    
run.complete()

In [ ]:
%%writefile $training_folder/register_diabetes.py
# ライブラリのインポート
import argparse
import joblib
from azureml.core import Workspace, Model, Run

# パラメータの取得
parser = argparse.ArgumentParser()
parser.add_argument('--model_folder', type=str, dest='model_folder', default="diabetes_model", help='model location')
args = parser.parse_args()
model_folder = args.model_folder

# 実験の実行コンテキストの取得
run = Run.get_context()

# モデルの読みこみ
print("Loading model from " + model_folder)
model_file = model_folder + "/model.pkl"
model = joblib.load(model_file)

Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'diabetes_model',
               tags={'Training context':'Pipeline'})

run.complete()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# クラスタが存在することを確認
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # もしない場合は作成
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4,
                                                           idle_seconds_before_scaledown=1800)
    pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

pipeline_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

# 実験用のPython環境を作る
diabetes_env = Environment("diabetes-pipeline-env")
diabetes_env.python.user_managed_dependencies = False # Azure MLに依存関係を管理させる
diabetes_env.docker.enabled = True # Dockerイメージを利用する

# パッケージの依存関係のセットを作成する
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','pandas'],
                                             pip_packages=['azureml-sdk'])

# 環境に依存関係を追加する
diabetes_env.python.conda_dependencies = diabetes_packages

# 環境を登録する (また利用するときのために)
diabetes_env.register(workspace=ws)
registered_env = Environment.get(ws, 'diabetes-pipeline-env')

# パイプライン用の新しいrunconfigオブジェクトを作成します。
pipeline_run_config = RunConfiguration()

# 上で作成したコンピュートを使用する
pipeline_run_config.target = pipeline_cluster

# 実行設定に環境を割り当てる
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

In [ ]:
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.estimator import Estimator

# トレーニングデータセットを取得する
#diabetes_ds = ws.datasets.get("diabetes dataset")

# モデルフォルダにPipelineData(一時データ参照)を作成します。
model_folder = PipelineData("model_folder", datastore=ws.get_default_datastore())

estimator = Estimator(source_directory=training_folder,
                        compute_target = pipeline_cluster,
                        environment_definition=pipeline_run_config.environment,
                        entry_script='diabetes_training.py')

# ステップ1, モデルを訓練するために推定器を実行する
train_step = EstimatorStep(name = "Train Model",
                           estimator=estimator, 
                           estimator_entry_script_arguments=['--output_folder', model_folder],
                           outputs=[model_folder],
                           compute_target = pipeline_cluster,
                           allow_reuse = True)

# ステップ2, モデル登録スクリプトを実行する
register_step = PythonScriptStep(name = "Register Model",
                                source_directory = training_folder,
                                script_name = "register_diabetes.py",
                                arguments = ['--model_folder', model_folder],
                                inputs=[model_folder],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# パイプラインの構成
pipeline_steps = [train_step, register_step]
pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
print("Pipeline is built.")

# 実験を作成してパイプラインを実行する
experiment = Experiment(workspace = ws, name = 'diabetes-training-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")

RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')